# Parking Tickets Analysis - Ann Arbor

This mini project analyzes parking ticket data from the city of Ann Arbor using Python and pandas. The data is obtained from FOIA requests and consists of multiple Excel files. Our analysis will include:

1. Loading and combining the ticket data
2. Generating descriptive statistics for ticket descriptions
3. Identifying the most common car make for tickets issued to NY plates
4. Analyzing the distribution of different types of Michigan license plates

## 1. Load and Combine Ticket Data

The `load_ticket_data()` function reads and combines data from multiple Excel files into a single pandas DataFrame. It handles different sheets within files and standardizes the column names.

In [1]:
import pandas as pd
import xlrd
import warnings

warnings.filterwarnings('ignore')

def load_ticket_data():
    def process_file(file):
        data = pd.read_excel(file, sheet_name=None, header=None)
        frames = []
        
        for sheet_name, df in data.items():
            print(f"Processing sheet: {sheet_name} from file: {file}")  # Debug print
            
            # Remove header rows and reset columns
            if not df.empty:
                df = df.iloc[4:]  # Adjusting for header rows
                df.columns = ['Ticket #', 'Badge', 'Issue Date', 'IssueTime', 'Plate', 'State', 
                              'Make', 'Model', 'Violation', 'Description', 'Location', 'Meter', 
                              'Fine', 'Penalty']
                
                # Remove footer row in the third sheet if present
                if sheet_name == 'Sheet3':
                    df = df[:-1]
                    
                frames.append(df)

        # Combine frames if not empty
        if frames:
            df_combined = pd.concat(frames, ignore_index=True)
            return df_combined

        return pd.DataFrame()  # Return empty df if no valid frames

    files = [
        'C:/Users/akama/Downloads/AnnArbor-TicketViolation2015.xls',
        'C:/Users/akama/Downloads/AnnArbor-TicketViolation2016.xls',
        'C:/Users/akama/Downloads/AnnArbor-TicketViolation2017.xls',
        'C:/Users/akama/Downloads/AnnArbor-TicketViolation2018.xls',
        'C:/Users/akama/Downloads/AnnArbor-TicketViolation2019.xls',
        'C:/Users/akama/Downloads/AnnArbor-TicketViolation-jan2020.xls'
    ]

    all_df = []
    for file in files:
        df = process_file(file)
        if not df.empty:
            all_df.append(df)
        else:
            print(f"No data found in file: {file}")  # Debug print

    if all_df:
        return pd.concat(all_df, ignore_index=True)
    else:
        raise ValueError("No data found in any files.")

# Load the data
df = load_ticket_data()
print(f"DataFrame Shape: {df.shape}")
df.head()

Processing sheet: Sheet1 from file: C:/Users/akama/Downloads/AnnArbor-TicketViolation2015.xls
Processing sheet: Sheet2 from file: C:/Users/akama/Downloads/AnnArbor-TicketViolation2015.xls
Processing sheet: Sheet3 from file: C:/Users/akama/Downloads/AnnArbor-TicketViolation2015.xls
Processing sheet: Sheet1 from file: C:/Users/akama/Downloads/AnnArbor-TicketViolation2016.xls
Processing sheet: Sheet2 from file: C:/Users/akama/Downloads/AnnArbor-TicketViolation2016.xls
Processing sheet: Sheet3 from file: C:/Users/akama/Downloads/AnnArbor-TicketViolation2016.xls
Processing sheet: Sheet1 from file: C:/Users/akama/Downloads/AnnArbor-TicketViolation2017.xls
Processing sheet: Sheet2 from file: C:/Users/akama/Downloads/AnnArbor-TicketViolation2017.xls
Processing sheet: Sheet3 from file: C:/Users/akama/Downloads/AnnArbor-TicketViolation2017.xls
Processing sheet: Sheet1 from file: C:/Users/akama/Downloads/AnnArbor-TicketViolation2018.xls
Processing sheet: Sheet2 from file: C:/Users/akama/Downloads

,Ticket #,Badge,Issue Date,IssueTime,Plate,State,Make,Model,Violation,Description,Location,Meter,Fine,Penalty
0,2110008524,826,2015-01-02 00:00:00,1719,DCM1327,MI,SATU,NaN,A01,EXPIRED METER,FARMER'S MARKET,17,10,0
1,2110008525,826,2015-01-02 00:00:00,1725,BAX385,IA,CHEV,NaN,A01,EXPIRED METER,FARMER'S MARKET,35,10,0
2,2100005834,821,2015-01-02 00:00:00,1344,2LEH1,MI,FORD,NaN,A04,NO PRKNG ANYTME,600 BLK OF WILLIAM E,NaN,25,0
3,2090006959,827,2015-01-02 00:00:00,1608,A380673,OH,FORD,NaN,A01,EXPIRED METER,200 BLK OF WASHINGTO,2029B,20,0
4,2100005829,821,2015-01-02 00:00:00,1326,ABD7951,AZ,HOND,NaN,A08,LOADING ZONE,1000 BLK OF UNIVERSI,NaN,35,0


## 2. Generate Descriptive Statistics for Ticket Descriptions

The `generate_descriptors(df)` function generates a DataFrame of unique ticket descriptions and their frequencies for different times of the day: morning (3 AM - 11:59 AM), afternoon (12 PM - 5:59 PM), and evening (6 PM - 2:59 AM).

In [2]:
def generate_descriptors(df):
    df['IssueTime'] = pd.to_numeric(df['IssueTime'], errors='coerce')
    df = df.dropna(subset=['IssueTime'])
    df['IssueTime'] = df['IssueTime'].astype(int)
    
    def get_period_count(df, start, end):
        return {desc: len(df[(df['Description'] == desc) & (df['IssueTime'] >= start) & (df['IssueTime'] < end)])
                for desc in df['Description'].dropna().unique()}
    
    morning_counts = get_period_count(df, 300, 1200)
    afternoon_counts = get_period_count(df, 1200, 1800)
    evening_counts = get_period_count(df, 1800, 2400) | get_period_count(df, 0, 300)
    
    return pd.DataFrame([morning_counts, afternoon_counts, evening_counts], 
                        index=["Morning", "Afternoon", "Evening"])

# Generate descriptors
df_descriptors = generate_descriptors(df)
print(df_descriptors.shape)
df_descriptors

(3, 51)


,EXPIRED METER,NO PRKNG ANYTME,LOADING ZONE,IMPROPER PARK,NO PERMIT CITY,PARKED ON WALK,15' FIRE HYDRAN,OTHER,LEFT TO CURB,HANDICAP,...,NO PARK STADIUM DAYS,ODD/EVEN PARK,OVER LEGAL TIME LIMIT,PKD IN INTERSECTION,IMPROPER (SIGNED),W/I 30' TRAF CONTROL,NO PKG HERE TO CORNER,ACROSS MARKED SPACES,UPON BRIDGE,BLOCKING TRAFFIC
Morning,186962,7866,1468,1763,889,732,1152,24965,613,2604,...,168,6,660,0,3,0,1,0,0,0
Afternoon,267256,16277,4093,2224,1683,1252,2182,31440,1031,4306,...,19,0,9520,2,0,5,2,1,1,1
Evening,18,147,1,9,0,4,10,105,11,125,...,0,8,0,0,0,0,0,0,0,0


## 3. Identify the Most Common Car Make for NY Plates

The `common_car_make()` function identifies the most common make of car that received tickets from the state of NY.

In [3]:
def common_car_make(df):
    ny_tickets = df[df['State'] == 'NY']
    most_common_make = ny_tickets['Make'].value_counts().idxmax()
    return most_common_make

# Identify the most common car make for NY plates
most_common_make_ny = common_car_make(df)
print(f"The most common car make for NY plates is: {most_common_make_ny}")

The most common car make for NY plates is: JEEP


## 4. Analyze Michigan Plates

The `fine_per_plates()` function analyzes the distribution of different types of Michigan license plates that received tickets.

In [4]:
import re

def fine_per_plates(df):
    mi_tickets = df[df['State'] == 'MI']
    plates = mi_tickets['Plate'].dropna()
    
    pattern_abc1234 = re.compile(r'^[A-Z]{3}[0-9]{4}$')
    pattern_abc123 = re.compile(r'^[A-Z]{3}[0-9]{3}$')
    pattern_123abc = re.compile(r'^[0-9]{3}[A-Z]{3}$')
    
    plate_counts = {
        "ABC1234": plates.str.contains(pattern_abc1234).sum(),
        "ABC123": plates.str.contains(pattern_abc123).sum(),
        "123ABC": plates.str.contains(pattern_123abc).sum()
    }
    plate_counts["vanity"] = len(plates) - sum(plate_counts.values())
    return plate_counts

# Analyze Michigan plates
mi_plate_distribution = fine_per_plates(df)
print(mi_plate_distribution)

{'ABC1234': 467758, 'ABC123': 42564, '123ABC': 83, 'vanity': 171266}


This brief project showcases my ability to manipulate and analyze large datasets using Python and pandas. The analyzed parking ticket data from Ann Arbor provides valuable insights into ticket trends and vehicle information. For further details, please check out my GitHub portfolio.